In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
train_data = pd.read_csv(r"D:\Projects\Spaceship-Titanic-ML\train.csv")# your path here
test_data = pd.read_csv(r"D:\Projects\Spaceship-Titanic-ML\test.csv")# your path here

y_train = train_data["Transported"]
features = train_data.drop("Transported", axis=1)
X_train = pd.get_dummies(features)
X_test = pd.get_dummies(test_data)

param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 3, 5, 7, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11, 13, 15],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [20, 30, 40],
    'p': [1, 2]
}

grid_search_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=10, scoring='accuracy')
grid_search_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=10, scoring='accuracy')

grid_search_rf.fit(X_train, y_train)
grid_search_knn.fit(X_train, y_train)

knn_best = grid_search_knn.best_estimator_
rf_best = grid_search_rf.best_estimator_

clf = VotingClassifier(
		estimators=[('knn_best', knn_best),('rf_best', rf_best)],
        voting='hard'
	)
clf.fit(X_train,y_train)
predictions = clf.predict(X_test)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Transported': predictions})
output.head(10)

In [ ]:
output.to_csv(r"D:\Projects\Spaceship-Titanic-ML\predictions.csv", index=False)# your path here